In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re

In [52]:
# Укажите путь к файлу в Google Drive
file_path = '/content/drive/MyDrive/train-00000-of-00001.parquet'

# Чтение файла
df = pd.read_parquet(file_path)

# Вывод первых строк
df.head()

image  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [53]:
# Список ненужных столбцов (замени на актуальные названия в твоём случае)
columns_to_drop = [
    'image', 'Uniq Id',
    'Model Number', 'Shipping Weight', 'Variants', 'Is Amazon Seller', 'Product Specification'
]

# Удалим столбцы, если они есть
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

In [54]:
# Отключаем перенос текста в ячейках (чтобы не было многоточий ...)
pd.set_option('display.width', None)  # Автоматическая ширина
pd.set_option('display.max_colwidth', None)  # Показывать полный текст в ячейках

In [55]:
df

,Product Name,Category,Selling Price,About Product,Technical Details,Product Url,description
0,"Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue",Toys & Games | Kids' Electronics | Electronic Learning Toys,$31.30,"Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!","Color:Blue show up to 2 reviews by default This sleek, hi-tech Bluetooth Smartwatch is made specifically for kids, and it's packed with apps and games for out-of-the-box fun! Take selfies and videos, play single and two-player games, message friends, listen to music, plus everything you need in a smartwatch– activity tracker, alarm/stopwatch, calculator, calendar and so much more! Plus, parents can add vital information like blood type and allergies to an 'in case of an emergency' (I. C. E. ) app | 7.2 ounces (View shipping rates and policies)",https://www.amazon.com/Kurio-Smartwatch-Bluetooth-Camera-Games/dp/B07TFD5D55,"Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow"
1,"Star Ace Toys Harry Potter & The Prisoner of Azkaban: Harry Potter with Dobby 1: 8 Scale Collectible Action Figure, Multicolor SA8011B",None,$174.99,"Make sure this fits by entering your model number. | From the classic film | Presents Harry as he appeared in the Prisoner Of Azkaban | Comes with the monster book of monsters, hedwig the owl, the Marauder's map and his wand | Includes a figure of the house Elf Dobby | Figure is in 1: 8 scale","From Star Ace Toys. Many fans would say that Harry Potter and the Prisoner Of Azkaban is their favorite film of the series. Directed by alfonso cuarón, this film is darker and more adult than the previous two films. Star ace is proud to present Harry Potter as he appeared in this movie in his Hogwarts school robes. He also comes with the monster book of monsters, Hedwig the owl, the Marauder's map and his wand. This special set also includes the house elf Dobby! | 1.43 pounds (View shipping rates and policies)",https://www.amazon.com/Star-Ace-Toys-Prisoner-Azkaban/dp/B07KMXGSXF,"Relive the magic! Star Ace Toys' 1/8 scale Harry Potter & Dobby collectible figure (SA8011B) from *Prisoner of Azkaban* is here. Highly detailed, this action figure features Harry and his loyal house-elf, Dobby. A must-have for Harry Potter collectors!"
2,Barbie Fashionistas Doll Wear Your Heart,Toys & Games | Dolls & Accessories | Dolls,$15.99,"Make sure this fits by entering your model number. | Barbie Fashionistas doll loves this outfit -- the pink Sweatshirt dress has a cool ""love"" typographic with a sheen touch | Boots and a choker complete the look | Her long hair is right on trend in high pigtails | More variety makes collecting Barbie Fashionistas dolls even more fun | Collect them all (each sold separately, subject to availability)","Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the ship method Ship it! | show up to 2 reviews by default Every Barbie fashionistas doll has her own look from cas

In [56]:
# Очистка Selling Price: удалим символы валют и запятые, приведём к float
df['Selling Price'] = (
    df['Selling Price']
    .astype(str)
    .replace(r'[₹$,]', '', regex=True)
    .replace(r',', '', regex=True)
    .astype(str)
    .str.strip()
)
df['Selling Price'] = pd.to_numeric(df['Selling Price'], errors='coerce')

In [57]:
df

,Product Name,Category,Selling Price,About Product,Technical Details,Product Url,description
0,"Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue",Toys & Games | Kids' Electronics | Electronic Learning Toys,31.30,"Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!","Color:Blue show up to 2 reviews by default This sleek, hi-tech Bluetooth Smartwatch is made specifically for kids, and it's packed with apps and games for out-of-the-box fun! Take selfies and videos, play single and two-player games, message friends, listen to music, plus everything you need in a smartwatch– activity tracker, alarm/stopwatch, calculator, calendar and so much more! Plus, parents can add vital information like blood type and allergies to an 'in case of an emergency' (I. C. E. ) app | 7.2 ounces (View shipping rates and policies)",https://www.amazon.com/Kurio-Smartwatch-Bluetooth-Camera-Games/dp/B07TFD5D55,"Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow"
1,"Star Ace Toys Harry Potter & The Prisoner of Azkaban: Harry Potter with Dobby 1: 8 Scale Collectible Action Figure, Multicolor SA8011B",None,174.99,"Make sure this fits by entering your model number. | From the classic film | Presents Harry as he appeared in the Prisoner Of Azkaban | Comes with the monster book of monsters, hedwig the owl, the Marauder's map and his wand | Includes a figure of the house Elf Dobby | Figure is in 1: 8 scale","From Star Ace Toys. Many fans would say that Harry Potter and the Prisoner Of Azkaban is their favorite film of the series. Directed by alfonso cuarón, this film is darker and more adult than the previous two films. Star ace is proud to present Harry Potter as he appeared in this movie in his Hogwarts school robes. He also comes with the monster book of monsters, Hedwig the owl, the Marauder's map and his wand. This special set also includes the house elf Dobby! | 1.43 pounds (View shipping rates and policies)",https://www.amazon.com/Star-Ace-Toys-Prisoner-Azkaban/dp/B07KMXGSXF,"Relive the magic! Star Ace Toys' 1/8 scale Harry Potter & Dobby collectible figure (SA8011B) from *Prisoner of Azkaban* is here. Highly detailed, this action figure features Harry and his loyal house-elf, Dobby. A must-have for Harry Potter collectors!"
2,Barbie Fashionistas Doll Wear Your Heart,Toys & Games | Dolls & Accessories | Dolls,15.99,"Make sure this fits by entering your model number. | Barbie Fashionistas doll loves this outfit -- the pink Sweatshirt dress has a cool ""love"" typographic with a sheen touch | Boots and a choker complete the look | Her long hair is right on trend in high pigtails | More variety makes collecting Barbie Fashionistas dolls even more fun | Collect them all (each sold separately, subject to availability)","Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the ship method Ship it! | show up to 2 reviews by default Every Barbie fashionistas doll has her own look from casual

In [58]:
nan_count = df.isna().sum()
print(f"Количество NaN в столбце: {nan_count}")

Количество NaN в столбце: Product Name           0
Category             120
Selling Price         63
About Product         35
Technical Details     92
Product Url            0
description            0
dtype: int64


In [59]:
# Сброс индекса (если нужно)
df = df.reset_index(drop=True)

# Векторизованное заполнение NaN
df['Selling Price'] = df['Selling Price'].fillna(
    (df['Selling Price'].ffill() + df['Selling Price'].bfill()) / 2
)
df['Selling Price'] = df['Selling Price'].fillna(df['Selling Price'].ffill())
df['Selling Price'] = df['Selling Price'].fillna(df['Selling Price'].bfill())

In [60]:
df = df.drop_duplicates()

In [61]:
df = df.dropna()

In [62]:
df

,Product Name,Category,Selling Price,About Product,Technical Details,Product Url,description
0,"Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue",Toys & Games | Kids' Electronics | Electronic Learning Toys,31.30,"Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!","Color:Blue show up to 2 reviews by default This sleek, hi-tech Bluetooth Smartwatch is made specifically for kids, and it's packed with apps and games for out-of-the-box fun! Take selfies and videos, play single and two-player games, message friends, listen to music, plus everything you need in a smartwatch– activity tracker, alarm/stopwatch, calculator, calendar and so much more! Plus, parents can add vital information like blood type and allergies to an 'in case of an emergency' (I. C. E. ) app | 7.2 ounces (View shipping rates and policies)",https://www.amazon.com/Kurio-Smartwatch-Bluetooth-Camera-Games/dp/B07TFD5D55,"Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow"
2,Barbie Fashionistas Doll Wear Your Heart,Toys & Games | Dolls & Accessories | Dolls,15.99,"Make sure this fits by entering your model number. | Barbie Fashionistas doll loves this outfit -- the pink Sweatshirt dress has a cool ""love"" typographic with a sheen touch | Boots and a choker complete the look | Her long hair is right on trend in high pigtails | More variety makes collecting Barbie Fashionistas dolls even more fun | Collect them all (each sold separately, subject to availability)","Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the ship method Ship it! | show up to 2 reviews by default Every Barbie fashionistas doll has her own look from casually cool to boho bold, all fashions are inspired by the latest trends. Collect them all to explore countless styles, fashions, shoes and accessories. The latest line of Barbie fashionistas dolls includes four body types, nine skin tones, 13 eye colors, 13 hairstyles and countless on-trend fashions and accessories. With these additions, girls everywhere will have infinitely more ways to play out their stories and spark their imaginations through Barbie -- because with Barbie, you can be anything! each sold separately, subject to availability. Dolls cannot stand alone. Clothing is designed to mix and match with dolls of the same body Type; select pieces can be shared across the line. Flat shoes fit dolls with articulated ankles or flat feet. Colors and decorations may vary. | 4.2 ounces (View shipping rates and policies)",https://www.amazon.com/Barbie-FJF44-Love-Fashion-Doll/dp/B0751ZH2ZT,"Express your style with Barbie Fashionistas Doll Wear Your Heart! This fashionable doll boasts a unique outfit and accessories, perfect for imaginative play. A great gift for kids aged 3+. Collect them all! #Barbie #Fashionistas #Doll #Toys #GirlsToys #FashionDoll #Play"
3,"Redcat Racing Aluminum Rear Lower Suspension Arms (2 Piece), Blue",Toys & Games | Hobbies | Remote & App Controll

In [63]:
#Импортируем TfIdfVectorizer из scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Определим векторизатор TF-IDF. Удалим все английские стоп-слова, такие как 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Все NaN заменим пустыми строками
df['description'] = df['description'].fillna('')

#Построми матрицу TF-IDF
tfidf_matrix = tfidf.fit_transform(df['description'])

#Выведем размерность этой матрицы
tfidf_matrix.shape

(1118, 5654)

In [64]:
from sklearn.metrics.pairwise import linear_kernel

# Вычисляем косинусное расстояние
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [65]:
df = df.rename(columns={"Product Name": "name"})

In [66]:
indices = pd.Series(df.index, index=df['name']).drop_duplicates()

In [67]:
def get_recommendations(name, cosine_sim=cosine_sim):
    idx = indices[name]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    product_indices = [i[0] for i in sim_scores]

    return df['name'].iloc[product_indices]

In [68]:
get_recommendations('Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue')

,name
216,"Rubies Monster High Fright Camera Action Draculaura Costume, Child Medium"
469,Rhode Island Novelty ~ 12 ~ Foam Dice ~ 1.5 Inch ~ Games Math Teaching ~ New
741,Junior Learning Touchtronic Placeholder Toy
1300,Swing-N-Slide Blue Child Seat
1092,Learning Resources Canadian Currency-X-Change
985,"Educational Insights Design & Drill See-Through Creative Workshop - Drill Toy, STEM & Construction, Building Toy"
695,"Kev & Cooper Playtime Collection Road Map Educational Area Rug - 3'3"" x 4'7"""
896,"Deny Designs Iveta Abolina Winter Blue Throw Pillow, Blue, Extra Large/26 x 26"
175,eKids Paw Patrol Bluetooth Portable MP3 Karaoke Machine Player with LED Display Store Hours of Music with Built in Memory Sing Along Using The Real Working Microphone USB Port to Expand Your Content
568,"Starplay Childrens Slide, Red/Blue"


In [69]:
# Объединение столбцов description и About Product
df['combined_text'] = df['description'] + " " + df['About Product']

# Объединяем название и категорию
df['name_category'] = df['name'] + ' ' + df['Category']

In [70]:
# Предобработка текста
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [71]:
def preprocess_text(text):
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Удаление пунктуации
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Лемматизация и удаление стоп-слов
    return ' '.join(words)

df['processed_text'] = df['combined_text'].apply(preprocess_text)
df['name_category'] = df['name_category'].apply(preprocess_text)

In [85]:
#Импортируем TfIdfVectorizer из scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Определим векторизатор TF-IDF. Удалим все английские стоп-слова, такие как 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Все NaN заменим пустыми строками
df['processed_text'] = df['processed_text'].fillna('')
df['name_category'] = df['name_category'].fillna('')

#Построми матрицу TF-IDF
tfidf_matrix = tfidf.fit_transform(df['processed_text'])
tfidf_matrix1 = tfidf.fit_transform(df['name_category'])

#Выведем размерность этой матрицы
tfidf_matrix.shape
tfidf_matrix1.shape

(1118, 4036)

In [96]:
indices = pd.Series(df.index, index=df['name_category']).drop_duplicates()

In [97]:
def get_recommendations(name, cosine_sim=cosine_sim):
    idx = indices[name]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    product_indices = [i[0] for i in sim_scores]

    return df['name_category'].iloc[product_indices]

In [99]:
get_recommendations('Redcat Racing Aluminum Rear Lower Suspension Arms (2 Piece), Blue Toys & Games | Hobbies | Remote & App Controlled Vehicles & Parts | Remote & App Controlled Vehicle Parts')

,name_category
966,"SunWorks Construction Paper, White, 9"" x 12"", 50 Sheets Arts, Crafts & Sewing | Crafting | Paper & Paper Crafts | Paper | Construction Paper"
161,"Creativity Street Dinosaurs Embossed Paper, 24-Pieces Toys & Games | Arts & Crafts"
338,"Graphic 45 4501829 Love Notes Stickers Craft Paper, Multi Toys & Games | Arts & Crafts | Stickers"
573,"Paper House Productions STM-0013E STM-0013 Unicorn Micro Stickers, 3-pack, Multi 3 Piece Toys & Games | Arts & Crafts | Stickers"
1158,"Mi-Zone Polka Dot Printed 100% Cotton Percale Ultra Soft 4 Piece Sheet Set for Girls Bedding, Full Size, Seafoam Home & Kitchen | Bedding | Kids' Bedding | Sheets & Pillowcases"
1303,VCNY Home Gangsta Shark Pillowcase and Sheet Sets Full Blue Home & Kitchen | Bedding | Kids' Bedding | Sheets & Pillowcases | Sheet & Pillowcase Sets
85,Essentials Sticker Toys & Games | Arts & Crafts | Stickers
1225,"Creativity Street Chenille Stems/Pipe Cleaners 12 Inch x 6mm 100-Piece, Assorted Colors Arts, Crafts & Sewing | Crafting | Craft Supplies | Pipe Cleaners"
1050,Nickelodeon Teenage Mutant Ninja Turtles 'Stomp Foot' Super Soft Microfiber Twin 3 Piece Sheet Set Home & Kitchen | Bedding | Kids' Bedding | Sheets & Pillowcases
1284,School Smart Chisel Tip Watercolor Markers - Set of 12 - Assorted Colors Toys & Games | Arts & Crafts


In [89]:
get_recommendations('Mexican Train Dominoes To Go')

,name
370,"MightySkins Skin Compatible with DJI Phantom 3 Standard – Be Happy | Protective, Durable, and Unique Vinyl Decal wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA"
1207,"MightySkins Skin Compatible with DJI Spark Mini Drone Combo - Style | Protective, Durable, and Unique Vinyl Decal wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA"
748,"MightySkins Skin Compatible with Swagtron T5 - Butterflies | Protective, Durable, and Unique Vinyl Decal wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA"
754,"MightySkins Skin Compatible with DJI Spark Mini Drone Combo - Music Man | Protective, Durable, and Unique Vinyl Decal wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA"
1165,"MightySkins Skin Compatible with DJI Inspire 1 Quadcopter Drone – Llama | Protective, Durable, and Unique Vinyl Decal wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA"
698,"MightySkins Skin Compatible with Parrot Bebop 2 - Black Zebra | Protective, Durable, and Unique Vinyl Decal wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA"
1041,"MightySkins Skin Compatible with Swagtron T5 - River Stones | Protective, Durable, and Unique Vinyl Decal wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA"
11,"MightySkins Skin Compatible with Swagtron T5 Hover Board – Deco Fish | Protective, Durable, and Unique Vinyl Decal wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA"
1096,"Skin Compatible with Razor A5 Prime Scooter - Peace Smile | MightySkins Protective, Durable, and Unique Vinyl Decal wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA"
503,"MightySkins Skin Compatible with Razor A2 Kick Scooter - Black Leather | Protective, Durable, and Unique Vinyl Decal wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA"


In [100]:
df.insert(0, 'id', None)
df


,id,name,Category,Selling Price,About Product,Technical Details,Product Url,description,combined_text,name_category,processed_text
0,None,"Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue",Toys & Games | Kids' Electronics | Electronic Learning Toys,31.30,"Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!","Color:Blue show up to 2 reviews by default This sleek, hi-tech Bluetooth Smartwatch is made specifically for kids, and it's packed with apps and games for out-of-the-box fun! Take selfies and videos, play single and two-player games, message friends, listen to music, plus everything you need in a smartwatch– activity tracker, alarm/stopwatch, calculator, calendar and so much more! Plus, parents can add vital information like blood type and allergies to an 'in case of an emergency' (I. C. E. ) app | 7.2 ounces (View shipping rates and policies)",https://www.amazon.com/Kurio-Smartwatch-Bluetooth-Camera-Games/dp/B07TFD5D55,"Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow","Kurio Glow Smartwatch: Fun, Safe & Educational! This kids' smartwatch boasts Bluetooth connectivity, built-in apps & games, and a camera – all in a vibrant blue design. Perfect for learning & play! #kidssmartwatch #kidselectronics #educationaltoys #kurioglow Make sure this fits by entering your model number. | Kurio watch glow is a real Bluetooth Smartwatch built especially for kids, packed with 20+ apps & games! | Get your glow on with new light-up feature that turns games and activities into colorful fun. | Kurio watch glow includes brand-new games with light effects, including the My little dragon virtual pet and color-changing mood sensor. | Play single and two-player games on one watch, Or connect two watches together via Bluetooth, plus motion-sensitive games that get kids moving! | Take fun selfies with the front-facing camera and decorate them with filters, frames and stickers. | Plus, everything you need in a smartwatch – activity tracker, music player, Alarm/stopwatch, calculator, calendar and so much more! | Scratch resistant and splash-proof - suitable for kids ages 4 and up!","Kurio Glow Smartwatch for Kids with Bluetooth, Apps, Camera & Games, Blue Toys & Games | Kids' Electronics | Electronic Learning Toys",kurio glow smartwatch fun safe educational kid smartwatch boast bluetooth connectivity builtin apps game camera vibrant blue design perfect learning play kidssmartwatch kidselectronics educationaltoys kurioglow make sure fit entering model number kurio watch glow real bluetooth smartwatch built especially kid packed apps game get glow new lightup feature turn game activity colorful fun kurio watch glow includes brandnew game light effect including little dragon virtual pet colorchanging mood sensor play single twoplayer game one watch connect two watch together via bluetooth plus motionsensitive game get kid moving take fun selfies frontfacing camera decorate filter frame sticker plus everything need smartwatch activity tracker m

In [102]:
from google.colab import files

# Сохраняем DataFrame во временный файл и скачиваем
df.to_csv('bd_vkr3.csv', index=False)
files.download('bd_vkr3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>